# Image Multimodal Search

This notebooks shows some preliminary work on Image Multimodal Search with lavis library. It is mainly meant to explore its capabilities and to decide on future research directions. We package our code into a `misinformation` package that is imported here:

In [ ]:
import misinformation
import misinformation.multimodal_search as ms

Set an image path as input file path.

In [ ]:
images = misinformation.utils.find_files(
    path="data/",
    limit=10,
)

In [ ]:
mydict = misinformation.utils.initialize_dict(images)

## Indexing and extracting features from images in selected folder

You can choose one of the following models: blip, blip2, albef, clip_base, clip_vitl14, clip_vitl14_336

In [ ]:
model_type = "blip"
# model_type = "blip2"
# model_type = "albef"
# model_type = "clip_base"
# model_type = "clip_vitl14"
# model_type = "clip_vitl14_336"

In [ ]:
(
    model,
    vis_processors,
    txt_processors,
    image_keys,
    image_names,
    features_image_stacked,
) = ms.MultimodalSearch.parsing_images(mydict, model_type, path_to_saved_tensors=".")

The tensors of all images `features_image_stacked` was saved in `<Number_of_images>_<model_name>_saved_features_image.pt`. If you run it once for current model and current set of images you do not need to repeat it again. Instead you can load this features with the command:

In [ ]:
# (
#    model,
#    vis_processors,
#    txt_processors,
#    image_keys,
#    image_names,
#    features_image_stacked,
# ) = ms.MultimodalSearch.parsing_images(mydict, model_type,"18_clip_base_saved_features_image.pt")

Here we already processed our image folder with 18 images with `clip_base` model. So you need just write the name `18_clip_base_saved_features_image.pt` of the saved file that consists of tensors of all images as a 3rd argument to the previous function. 

Next, you need to form search queries. You can search either by image or by text. You can search for a single query, or you can search for several queries at once, the computational time should not be much different. The format of the queries is as follows:

In [ ]:
search_query3 = [
    {"text_input": "politician press conference"},
    {"text_input": "a person wearing a mask"},
    {"image": "data/106349S_por.png"},
]

You can filter your results in 3 different ways:
- `filter_number_of_images` limits the number of images found. That is, if the parameter `filter_number_of_images = 10`, then the first 10 images that best match the query will be shown. The other images ranks will be set to `None` and the similarity value to `0`.
- `filter_val_limit` limits the output of images with a similarity value not bigger than `filter_val_limit`. That is, if the parameter `filter_val_limit = 0.2`, all images with similarity less than 0.2 will be discarded.
- `filter_rel_error` (percentage) limits the output of images with a similarity value not bigger than `100 * abs(current_simularity_value - best_simularity_value_in_current_search)/best_simularity_value_in_current_search < filter_rel_error`. That is, if we set filter_rel_error = 30, it means that if the top1 image have 0.5 similarity value, we discard all image with similarity less than 0.35.

In [ ]:
similarity = ms.MultimodalSearch.multimodal_search(
    mydict,
    model,
    vis_processors,
    txt_processors,
    model_type,
    image_keys,
    features_image_stacked,
    search_query3,
)

After launching `multimodal_search` function, the results of each query will be added to the source dictionary.  

In [ ]:
mydict["106349S_por"]

A special function was written to present the search results conveniently. 

In [ ]:
ms.MultimodalSearch.show_results(mydict, search_query3[0])

## Save search results to csv

Convert the dictionary of dictionarys into a dictionary with lists:

In [ ]:
outdict = misinformation.utils.append_data_to_dict(mydict)
df = misinformation.utils.dump_df(outdict)

Check the dataframe:

In [ ]:
df.head(10)

Write the csv file:

In [ ]:
df.to_csv("./data_out.csv")